In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
from torchvision import models
from torch import nn

In [ ]:
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=1, shuffle=True)

In [ ]:
model = models.densenet121(pretrained=True)
model.classifier = nn.Linear(model.classifier.in_features, 10)
model.eval()

In [ ]:
#!pip install captum

import torch
import matplotlib.pyplot as plt
from captum.attr import LayerGradCam, LayerAttribution

images, labels = next(iter(testloader))
input_img = images
label = labels.item()

target_layer = model.features[-1]
gradcam = LayerGradCam(model, target_layer)
attributions = gradcam.attribute(input_img, target=label)
upsampled_attr = LayerAttribution.interpolate(attributions, input_img.shape[2:])

def show_saliency_on_image(img, saliency, title=""):
    img = img.squeeze().permute(1,2,0).detach().cpu().numpy()
    img = (img * 0.5) + 0.5
    saliency = saliency.squeeze().detach().cpu().numpy()
    if saliency.ndim == 3:
        saliency = saliency.mean(axis=0)
    plt.figure(figsize=(8,4))
    plt.subplot(1,2,1)
    plt.imshow(img)
    plt.title("Immagine originale")
    plt.axis('off')
    plt.subplot(1,2,2)
    plt.imshow(img, alpha=0.5)
    plt.imshow(saliency, cmap='hot', alpha=0.5)
    plt.title(title)
    plt.axis('off')
    plt.show()

show_saliency_on_image(input_img, upsampled_attr, title="Grad-CAM")

In [ ]:
# 1. Downgrade numpy
#!pip install numpy==1.23.5 --force-reinstall

# 2. Reinstalla lime
#!pip install lime --force-reinstall
from lime import lime_image
from skimage.segmentation import mark_boundaries

def batch_predict(images):
    model.eval()
    batch = torch.stack([transform(transforms.ToPILImage()(img)) for img in images], dim=0)
    logits = model(batch)
    probs = torch.nn.functional.softmax(logits, dim=1)
    return probs.detach().numpy()

explainer = lime_image.LimeImageExplainer()
img = input_img.squeeze().permute(1,2,0).numpy()
explanation = explainer.explain_instance(img, batch_predict, top_labels=1, hide_color=0, num_samples=1000)
temp, mask = explanation.get_image_and_mask(label, positive_only=True, num_features=5, hide_rest=False)
plt.imshow(mark_boundaries(temp / 2 + 0.5, mask))
plt.title("LIME")
plt.axis('off')
plt.show()

In [ ]:
from captum.attr import Occlusion

occlusion = Occlusion(model)
attr_occ = occlusion.attribute(
    input_img,
    strides=(3, 8, 8),
    target=label,
    sliding_window_shapes=(3, 15, 15)
)

show_saliency_on_image(input_img, attr_occ, title="Occlusion")

In [ ]:
print("MNIST: 70.000 immagini di cifre scritte a mano (0-9), 28x28 pixel, 1 canale (convertito a 3 per DenseNet).")

In [ ]:
for images, labels in testloader:
    outputs = model(images)
    _, preds = torch.max(outputs, 1)
    if preds != labels:
        print(f"Errore: Predetto {preds.item()}, Reale {labels.item()}")
        break

In [ ]:
def simple_rule_based_classifier(img):
    img = img.squeeze().numpy()
    if img.sum() < 100:
        return 1
    else:
        return 0

In [ ]:
errore_img = None
errore_label = None
errore_pred = None

for images, labels in testloader:
    with torch.no_grad():
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        if preds != labels:
            print(f"Errore: Predetto {preds.item()}, Reale {labels.item()}")
            errore_img = images
            errore_label = labels.item()
            errore_pred = preds.item()
            break

In [ ]:
from torchvision import models
from torch import nn
from captum.attr import LayerGradCam, LayerAttribution, Occlusion

model = models.densenet121(pretrained=True)
model.classifier = nn.Linear(model.classifier.in_features, 10)
model.eval()

target_layer = model.features[-1]
gradcam = LayerGradCam(model, target_layer)
occlusion = Occlusion(model)

attributions = gradcam.attribute(errore_img, target=errore_label)
upsampled_attr = LayerAttribution.interpolate(attributions, errore_img.shape[2:])
show_saliency_on_image(errore_img, upsampled_attr, title=f"Grad-CAM (Predetto {errore_pred}, Reale {errore_label})")

attr_occ = occlusion.attribute(
    errore_img,
    strides=(3, 8, 8),
    target=errore_label,
    sliding_window_shapes=(3, 15, 15)
)
show_saliency_on_image(errore_img, attr_occ, title=f"Occlusion (Predetto {errore_pred}, Reale {errore_label})")